In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
import  os
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
es = Elasticsearch("http://localhost:9200")
output_columns = ["Компания",	
                  "Год счета",	
                  "Номер счета",	
                  "Позиция счета",	
                  "Номер позиции распределения",	
                  "Дата отражения в учетной системе",	
                  "ID договора",	
                  "Услуга",	
                  "Класс услуги",	
                  "Здание",	
                  "Класс ОС",	
                  "ID основного средства",	
                  'Признак "Использование в основной деятельности"',	
                  'Признак "Способ использования"',	
                  "Площадь",	
                  "Счет главной книги"
                  ]

In [33]:
def join_data(path_to_bills: str):
    output_df = pd.DataFrame(columns=output_columns)
    bill_df = pd.read_excel(path_to_bills)
    bill_df.drop(["Стоимость без НДС"], axis=1)
    columns_list = bill_df.columns
    for index, row in bill_df.iterrows():
        new_df = pd.DataFrame(output_columns)
        #print(row["ID договора"])
        all_docs = es.search(
            index="связь_договор_-_здания",
            query= {
                "bool": {
                    "must": {
                        "match_phrase": {
                            "ID договора": row["ID договора"]
                        }
                    }
                }
            }
        )
        if len(all_docs["hits"]["hits"]) > 0:
            logging.info(all_docs)
            for doc in all_docs["hits"]["hits"]:
                for column in columns_list:
                    new_df[column] = row[column]

                # поиск класса услуги по ее id
                resp = es.search(
                index="коды_услуг",
                query={
                        "bool": {
                            "must": {
                                "match_phrase": {
                                    "ID услуги": row["ID услуги"],
                                }
                            },
                        },
                    },            
                )
                new_df["Класс услуги"] = resp.body["hits"]["hits"][0]["_source"]["Класс услуги"]
                logging.info(doc)
                new_df["ID здания"] = doc["_source"]["ID здания"]
                resp = es.search(
                index="основные_средства",
                query={
                        "bool": {
                            "must": {
                                "match_phrase": {
                                    "ID здания": doc["_source"]["ID здания"],
                                    'Признак "Используется в основной деятельности"': row['Признак "Используется в основной деятельности"'],
                                    'Признак "Способ использования"': row['Признак "Способ использования"']
                                }
                            },
                        },
                    },            
                ) 
                print(resp.body["hits"]["hits"][0]["_source"])
                new_df["ID основного средства"] = resp.body["hits"]["hits"][0]["_source"]["ID основного средства"]
            return new_df

            # поиск id здания по id договора

        






In [34]:
new_df = join_data("../data/bills/Счета на оплату 3800-2023.XLSX")
print(new_df.head(1))

2024-06-08 01:39:29,603 INFO POST http://localhost:9200/%D1%81%D0%B2%D1%8F%D0%B7%D1%8C_%D0%B4%D0%BE%D0%B3%D0%BE%D0%B2%D0%BE%D1%80_-_%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F/_search [status:200 duration:0.005s]
C:\Users\Terri\AppData\Local\Temp\ipykernel_16520\3452721985.py:9: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  all_docs = es.search(
2024-06-08 01:39:29,628 INFO POST http://localhost:9200/%D1%81%D0%B2%D1%8F%D0%B7%D1%8C_%D0%B4%D0%BE%D0%B3%D0%BE%D0%B2%D0%BE%D1%80_-_%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F/_search [status:200 duration:0.005s]
2024-06-08 01:39:29,701 INFO POST http://localhost:9200/%D1%81%D0%B2%D1%8F%D0%B7%D1%8C_%D0%B4%D0%BE%D0%B3%D0%BE%D0%B2%D0%BE%D1%80_-_%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F/_search [status:200 duration:0.045s]
2024-06-08 01:39:29,726 INFO P

KeyError: 'Признак "Используется в основной деятельности"'